# fuzzy-sl Wikibase Archaeological Sites from the Silver Coinage of Cronon as Map

This notebook fetches archaeological sites using a SPARQL query from the fuzzy-sl Wikibase and creates maps.

## Define SPARQL query service

In [23]:
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
from matplotlib.patches import Patch
from scipy.stats import gaussian_kde
import numpy as np

def querySparql(query):
    sparql = SPARQLWrapper("https://fuzzy-sl.wikibase.cloud/query/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

## Define the SPARQL Query

In [24]:
# SPARQL Query
sitesQuery = """
PREFIX fslwd: <https://fuzzy-sl.wikibase.cloud/entity/>
PREFIX fslwdt: <https://fuzzy-sl.wikibase.cloud/prop/direct/>
PREFIX fslpq: <https://fuzzy-sl.wikibase.cloud/prop/qualifier/>

SELECT ?item ?itemLabel ?geo WHERE { 
  ?item fslwdt:P1 fslwd:Q10 .
  ?item fslwdt:P3 fslwd:Q14 .
  ?item fslwdt:P4 ?geo .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
"""

## Fetch Data and Convert to DataFrame

In [25]:
# Fetch data using the SPARQL query
sparql_results = querySparql(sitesQuery)

# Convert SPARQL JSON results into a DataFrame
data = []
for result in sparql_results:
    geo = result['geo']['value'] if 'geo' in result else None
    lat, lon = (None, None)
    if geo:
        lon, lat = map(float, geo.replace("Point(", "").replace(")", "").split())
    data.append({
        "item": result['item']['value'],
        "itemLabel": result['itemLabel']['value'],
        "latitude": lat,
        "longitude": lon,
    })

df = pd.DataFrame(data)
df

""


## Visualise the Data in a map

In [26]:
# Check if DataFrame is populated
if df.empty:
    print("No data retrieved from the query.")
else:
    # Filter rows with valid coordinates
    df_with_coords = df.dropna(subset=['latitude', 'longitude'])

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df_with_coords,
        geometry=[Point(xy) for xy in zip(df_with_coords['longitude'], df_with_coords['latitude'])],
        crs="EPSG:4326"
    )

    # Convert to Web Mercator for OSM basemap
    gdf_mercator = gdf.to_crs(epsg=3857)

    # Plot points on the map
    fig, ax = plt.subplots(figsize=(12, 8))
    gdf_mercator.plot(ax=ax, color='red', markersize=50, alpha=0.7, label="archaeological sites")
    
    # Add OSM basemap
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    
    ax.set_axis_off()
    plt.title("Map of archaeological sites in Chieti")
    plt.legend()
    plt.show()

No data retrieved from the query.
